In [59]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature
from sklearn.datasets import load_iris

In [60]:
## load the dataset 
data= datasets.load_iris()
feature_values=data.data
target_values=data.target
feature_names = datasets.load_iris().feature_names
df = pd.DataFrame(feature_values, columns=feature_names)
df['target']= target_values
df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [61]:
## split the data into training and test sets 
X=df.drop('target',axis='columns')
y=df['target']
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20)

## Define the model hyperparameters
params={"penalty":"l2","solver":"lbfgs","max_iter":1000,"multi_class":"auto","random_state":8888}
lr=LogisticRegression(**params)
lr.fit(X_train,y_train)

c:\Users\punee\anaconda3\envs\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [62]:
## Prediction on the test set 
y_pred=lr.predict(X_test)


In [63]:
# Create DataFrame to compare actual vs predicted labels
df_predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Map numeric labels to species names
df_predictions['Actual Species'] = df_predictions['Actual'].map(lambda x: data.target_names[x])
df_predictions['Predicted Species'] = df_predictions['Predicted'].map(lambda x: data.target_names[x])
df_predictions.head()

,Actual,Predicted,Actual Species,Predicted Species
134,2,2,virginica,virginica
47,0,0,setosa,setosa
73,1,1,versicolor,versicolor
99,1,1,versicolor,versicolor
128,2,2,virginica,virginica


In [64]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9666666666666667

MLFLOW Tracking

In [65]:
##MLFLOW Tracking  
mlflow.set_tracking_uri("http://127.0.0.1:5000")
## create new mlflow experiment 
mlflow.set_experiment("MLFLOW Quickstart")
## Start a MLFLOW run()
with mlflow.start_run():
    mlflow.log_params(params)
    ##Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)
    ## Set a tag that we can use to remind ourselves what this run was for 
    mlflow.set_tag("Training Info","Basic LR model for iris data")
    ## Infer the model signature
    signature=infer_signature(X_train, lr.predict(X_train))

    ##log the model 
    model_info= mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
        
    )




Successfully registered model 'tracking-quickstart'.
2025/02/12 11:09:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run overjoyed-fox-152 at: http://127.0.0.1:5000/#/experiments/678528034111962380/runs/aa14ac96a93446a0aa63e2e862f6994a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/678528034111962380


Created version '1' of model 'tracking-quickstart'.


## Making Hyparameter Tunning

In [66]:
params={"solver":"lbfgs","max_iter":1000,"multi_class":"auto","random_state":88}
lr=LogisticRegression(**params)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)

c:\Users\punee\anaconda3\envs\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [67]:
##MLFLOW Tracking  
mlflow.set_tracking_uri("http://127.0.0.1:5000")
## create new mlflow experiment 
mlflow.set_experiment("MLFLOW Quickstart")
## Start a MLFLOW run()
with mlflow.start_run():
    mlflow.log_params(params)
    ##Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)
    ## Set a tag that we can use to remind ourselves what this run was for 
    mlflow.set_tag("Training Info","Basic LR model for iris data")
    ## Infer the model signature
    signature=infer_signature(X_train, lr.predict(X_train))

    ##log the model 
    model_info= mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
        
    )




Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/02/12 11:09:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2


🏃 View run beautiful-finch-369 at: http://127.0.0.1:5000/#/experiments/678528034111962380/runs/9ea5e1d7f69945469776d4f751e3ba21
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/678528034111962380


Created version '2' of model 'tracking-quickstart'.


Inferencing and Validating the Model : LOAD THE MODEL BACK FOR PREDICTION AS GENERIC PYTHON FUNCTION MODEL 

In [68]:
import mlflow

model_uri = 'runs:/d3891da37b4f49de9bdfa3747fc6aab2/iris_model'
# This is the input example logged with the model
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example
print(input_data)

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="local",
)

2025/02/12 11:09:53 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.


     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  6.4               3.1                5.5               1.8
1                  5.1               3.7                1.5               0.4
2                  5.8               2.7                4.1               1.0
3                  4.4               3.0                1.3               0.2
4                  5.5               2.4                3.8               1.1
..                 ...               ...                ...               ...
115                6.7               3.0                5.0               1.7
116                4.8               3.0                1.4               0.1
117                6.5               2.8                4.6               1.5
118                5.3               3.7                1.5               0.2
119                6.4               2.8                5.6               2.1

[120 rows x 4 columns]


2025/02/12 11:09:54 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


{"predictions": [2, 0, 1, 0, 1, 0, 0, 0, 1, 2, 1, 0, 2, 1, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 0, 2, 2, 0, 0, 1, 1, 0, 1, 1, 2, 2, 0, 1, 2, 2, 2, 0, 0, 2, 1, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0, 0, 0, 1, 2, 1, 0, 2, 0, 2, 1, 2, 2, 0, 0, 1, 1, 1, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 1, 0, 1, 0, 2]}

Model Registry

THE MLFLOW MODEL REGISTRY COMPONENT IS A CENTRALIZED MODEL STORE, SET OF APIs, and UI, to collaboratively manage the full lifecycle of an MLflow model. it provides model lineage (which MLflow experiment and run produced the model),model versioning,model aliasing, model tagging and annotations

In [69]:
### Without Registering 
##MLFLOW Tracking  
mlflow.set_tracking_uri("http://127.0.0.1:5000")
## create new mlflow experiment 
mlflow.set_experiment("MLFLOW Quickstart")
## Start a MLFLOW run()
with mlflow.start_run():
    mlflow.log_params(params)
    ##Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)
    ## Set a tag that we can use to remind ourselves what this run was for 
    mlflow.set_tag("Training Info","Basic LR model for iris data")
    ## Infer the model signature
    signature=infer_signature(X_train, lr.predict(X_train))

    ##log the model 
    model_info= mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        
    )


🏃 View run lyrical-toad-832 at: http://127.0.0.1:5000/#/experiments/678528034111962380/runs/294fb773c90642b6b17cf30ca8ed9787
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/678528034111962380


## INFERENCING MODEL FROM MODEL REGISTRY

In [ ]:
## INFERENCING MODEL FROM MODEL REGISTRY

import mlflow.sklearn
model_name="tracking-quickstart"
model_version="latest"

model_uri=f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)


c:\Users\punee\anaconda3\envs\venv\lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


LogisticRegression(max_iter=1000, multi_class='auto', random_state=88)

In [71]:
model_uri

'models:/tracking-quickstart/latest'

In [ ]:
y_pred_new = model.predict(X_test)